# 필요한 라이브러리 호출

In [2]:
!pip install geopandas

     |████████████████████████████████| 994 kB 7.0 MB/s 
     |████████████████████████████████| 15.4 MB 39 kB/s 
     |████████████████████████████████| 6.3 MB 25.0 MB/s 


In [3]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import platform
from matplotlib import font_manager, rc
import pandas as pd
import folium
import branca

if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')

In [4]:
from shapely.geometry.point import Point

# 서울시 AED 좌표를 불러와 관악구 데이터만 가져오기

In [6]:
pd_geo_coordi = pd.read_csv('../../AED_Seoul_coord.csv', engine='python', encoding='CP949')

FileNotFoundError: ignored

In [ ]:
aed_gwanak = pd_geo_coordi.loc[pd_geo_coordi['도로명'].str.split().str.get(1)=='관악구']

In [ ]:
lats = np.array(aed_gwanak['위도'])
lons = np.array(aed_gwanak['경도'])

In [ ]:
lat_mean, lon_mean = (37.472, 126.95)

In [ ]:
# import math
# R = 6378
# r = R * math.cos(lat_mean*math.pi/180)
# dtheta = (0.3 / r) * (180/math.pi)
# dtheta

In [ ]:
# def make_circle(point, radius = dtheta):
#     point = Point(point)
#     return point.buffer(radius, resolution=100)

In [ ]:
aed_gwanak

# 각 AED로부터 300m 원을 이루는 polygon 생성

In [ ]:
aed_gwanak['geom'] = aed_gwanak.apply(lambda r: Point(r['경도'], r['위도']), axis=1)

In [ ]:
aed_gwanak

In [ ]:
gdf = gpd.GeoDataFrame(aed_gwanak, geometry='geom', crs='epsg:4326')
gdf_flat = gdf.to_crs('epsg:6347')
gdf_flat['geom'] = gdf_flat.geometry.buffer(300)
gdf = gdf_flat.to_crs('epsg:4326')

In [ ]:
gdf

In [ ]:
u = gdf.iloc[0, 4]

In [ ]:
u

# AED 반경 300m로 생성된 polygon을 모두 union

In [ ]:
for _, r in gdf.iterrows():
    u = u.union(r['geom'])

In [ ]:
u

In [ ]:
# u = make_circle((lons[0], lats[0]))

In [ ]:
# for i in range(1, len(lats)):
#     u2 = make_circle((lons[i], lats[i]))
#     u = u.union(u2)

In [ ]:
# u

# folium을 통해 지도상에 300m 원 표시

In [ ]:
m2 = folium.Map([lat_mean, lon_mean], zoom_start = 14)

In [ ]:
for lat, lon in zip(lats, lons):
    try:
        folium.Circle(location = [lat, lon], radius = 300, color='black', fill_color = 'black').add_to(m2) # 반경 300m 원 그리기
    except:
        continue

In [ ]:
m2

# 20대 남자 인구 데이터를 기반으로 하여 각 grid별로 AED가 커버하지 못하는 비율 계산

In [ ]:
base = "./20남/nlsp_021001026.shp"
df = gpd.read_file(base, encoding='utf8')
df

추후 OHCR 계산 편의를 위해 m20 column 생성, 불필요한 columns 제거

In [ ]:
df['m20'] = df['val']
df

In [ ]:
df = df.drop(columns=['lbl', 'val'])
df

polygon을 좌표평면계로 변경

In [ ]:
df = df.to_crs(epsg=4326)
df

NaN을 0으로 대체

In [ ]:
df = df.fillna(0)
df

## 각 polygon끼리 겹치는 넓이 비율 계산 함수 생성

In [ ]:
def cal_area(poly, aed):
    area = poly.area
    intersection = poly.intersection(aed).area
    return intersection / area

# 지도에 표시

In [ ]:
m3 = folium.Map([lat_mean, lon_mean], zoom_start = 14)

In [ ]:
for _, r in df.iterrows():
    value = 1-cal_area(r['geometry'], u)
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()

    color = plt.cm.OrRd(value)

    color = mpl.colors.to_hex(color)
    
#     geo_j = folium.GeoJson(data=geo_j,
#                            style_function=lambda x: {'fillColor': 'orange'})
    geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.8,
                                                                    })


    folium.Popup(f'{value:.3f}').add_to(geo_j)
    geo_j.add_to(m3)

In [ ]:
colormap = branca.colormap.linear.OrRd_06.scale(0, 1)
colormap = colormap.to_step(index=np.linspace(0, 1, 1000))
colormap.caption = 'Area Ratio NOT Covered by AEDs (300m)'
colormap.add_to(m3)
m3

# 119 구조센터

In [ ]:
fs = pd.read_csv('../../FS_Seoul_coord.csv', encoding='cp949')

In [ ]:
fs_gwanak = fs.loc[fs['위치'].str.split().str.get(1)=='관악구']
fs_gwanak

In [ ]:
fs['geom'] = fs.apply(lambda r: Point(r['경도'], r['위도']), axis=1)

In [ ]:
fs

In [ ]:
fs_gdf = gpd.GeoDataFrame(fs, geometry='geom', crs='epsg:4326')
fs_gdf_flat = fs_gdf.to_crs('epsg:6347')
fs_gdf_flat['geom'] = fs_gdf_flat.geometry.buffer(1000)
fs_gdf = fs_gdf_flat.to_crs('epsg:4326')

In [ ]:
fslats = np.array(fs['위도'])
fslons = np.array(fs['경도'])

In [ ]:
for lat, lon in zip(fslats, fslons):
    try:
        folium.Circle(location = [lat, lon], radius = 1000, color='black', fill_color = 'black').add_to(m2) # 반경 300m 원 그리기
    except:
        continue

In [ ]:
m2

In [ ]:
# R = 6378
# r = R * math.cos(lat_mean*math.pi/180)
# radius = (1 / r) * (180/math.pi)
# u3 = u
# for i in range(len(fslats)):
#     u2 = make_circle((fslons[i], fslats[i]), radius=radius)
#     u3 = u3.union(u2)

## 119 구조센터 1km + AED 300m

In [ ]:
u2 = u
for _, r in fs_gdf.iterrows():
    u2 = u2.union(r['geom'])

In [ ]:
m3 = folium.Map([lat_mean, lon_mean], zoom_start = 14)

In [ ]:
for _, r in df.iterrows():
    value = 1-cal_area(r['geometry'], u2)
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()

    color = plt.cm.OrRd(value)

    color = mpl.colors.to_hex(color)
    
#     geo_j = folium.GeoJson(data=geo_j,
#                            style_function=lambda x: {'fillColor': 'orange'})
    geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.8,
                                                                    })


    folium.Popup(f'{value:.3f}').add_to(geo_j)
    geo_j.add_to(m3)

In [ ]:
colormap = branca.colormap.linear.OrRd_06.scale(0, 1)
colormap = colormap.to_step(index=np.linspace(0, 1, 1000))
colormap.caption = 'Area Ratio NOT Covered by AEDs and 119 stations'
colormap.add_to(m3)
m3

# 20대 남자~ 100세 이상 여성 데이터 불러오기

In [ ]:
filenumberlist = []
for i in range(1026, 1051+1):
    if i % 3 != 2:
        filenumberlist.append(i)

In [ ]:
for i in range(18):
    ageband = (i // 2 + 2) * 10
    filenumber = filenumberlist[i]
    if i % 2 == 0:
        sex = '남'
        sexa = 'm'
    else:
        sex = '여'
        sexa = 'f'
    file = f'./{ageband}{sex}/nlsp_02100{filenumber}.shp'
    age = gpd.read_file(file, encoding='utf8')
    age = age[['gid', 'val']]
    age = age.rename(columns={'val':f'{sexa}{ageband}'})
    df = pd.merge(df, age, on='gid', how='left')

In [ ]:
df = df.drop(columns=['m20_x'])
df = df.rename(columns={'m20_y':'m20'})

In [ ]:
df

In [ ]:
df = df.fillna(0)
df

## 심정지 통계 자료에 맞춰 80세 이상으로 묶음

In [ ]:
df['m80+'] = df['m80'] + df['m90'] + df['m100'] 
df['f80+'] = df['f80'] + df['f90'] + df['f100']

In [ ]:
df.drop(columns=['m80', 'f80', 'm90', 'f90', 'm100', 'f100'], inplace=True)

In [ ]:
df

In [ ]:
d = pd.read_csv('../심정지발생률.csv', encoding='cp949')

In [ ]:
d

In [ ]:
d = d.drop(index=[3, 13])

In [ ]:
d['발생건수 (건)'] = pd.to_numeric(d['발생건수 (건)'])
d['발생률 (인구 10만명당 명)'] = pd.to_numeric(d['발생률 (인구 10만명당 명)'])

In [ ]:
d

## 강남구 논문의 OHCA 식으로 계산
$OC_i = R_m \sum_{j=1}^kBR_j \times PM_j + R_f \sum_{j=1}^kBR_j \times PF_j$  
$OC_i$ : 공간 단위 i에서 추정된 심정지 가능 인구  
$j$ : 연령 그룹 인덱스  
$BR_j$ : 연령 그룹 $j$의 심정지 발생률  
$PM_j, PF_j$ : 각각 연령 그룹 $j$의 남성, 여성 인구  
$R_m, R_f$ : 각각 관측된 심정지 환자에서 남성과 여성의 비율

In [ ]:
rm = d.iloc[1, 1] / d.iloc[0, 1]
rf = d.iloc[2, 1] / d.iloc[0, 1]
rm, rf

In [ ]:
female = df.iloc[:, [i for i in range(2, 16) if i%2 == 1]]
male = df.iloc[:, [i for i in range(2, 16) if i%2 == 0]]

In [ ]:
female

In [ ]:
df['OHCA'] = 0
df['brpm'] = 0
df['brpf'] = 0
df

In [ ]:
for i in range(7):
    df['brpm'] = df['brpm'] + d.iloc[i+5, 2] * male.iloc[:, i]
    df['brpf'] = df['brpf'] + d.iloc[i+5, 2] * female.iloc[:, i]
df['OHCA'] = (rm * df['brpm'] + rf * df['brpf']) / 100000

In [ ]:
df

In [ ]:
max_ohca = df['OHCA'].max()

In [ ]:
m4 = folium.Map([lat_mean, lon_mean], zoom_start = 14)

In [ ]:
for _, r in df.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    value = r['OHCA']
    color = plt.cm.OrRd(value/max_ohca)

    color = mpl.colors.to_hex(color)
    
#     geo_j = folium.GeoJson(data=geo_j,
#                            style_function=lambda x: {'fillColor': 'orange'})
    geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.8,
                                                                    })


    folium.Popup(f'{value:.3f}').add_to(geo_j)
    geo_j.add_to(m4)

In [ ]:
colormap = branca.colormap.linear.OrRd_06.scale(0, max_ohca)
colormap = colormap.to_step(index=np.linspace(0, max_ohca, 1000))
colormap.caption = 'OHCA'
colormap.add_to(m4)
m4

In [ ]:
m5 = folium.Map([lat_mean, lon_mean], zoom_start = 14)

In [ ]:
values = []
for _, r in df.iterrows():
    value = (1-cal_area(r['geometry'], u)) * r['OHCA']
    values.append(value)
max_value = max(values)
for i, r in df.iterrows():
    value = values[i]
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()

    color = plt.cm.OrRd(value / max_value)

    color = mpl.colors.to_hex(color)
    
#     geo_j = folium.GeoJson(data=geo_j,
#                            style_function=lambda x: {'fillColor': 'orange'})
    geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.8,
                                                                    })


    folium.Popup(f'{value:.3f}').add_to(geo_j)
    geo_j.add_to(m5)

In [ ]:
colormap = branca.colormap.linear.OrRd_06.scale(0, max_value)
colormap = colormap.to_step(index=np.linspace(0, max_value, 1000))
colormap.caption = 'OHCA × Area Ratio not covered by AEDs'
colormap.add_to(m5)
m5

In [ ]:
m6 = folium.Map([lat_mean, lon_mean], zoom_start = 14)

In [ ]:
values = []
for _, r in df.iterrows():
    value = (1-cal_area(r['geometry'], u2)) * r['OHCA']
    values.append(value)
max_value = max(values)
for i, r in df.iterrows():
    value = values[i]
    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()

    color = plt.cm.Reds(value/max_value)

    color = mpl.colors.to_hex(color)
    
#     geo_j = folium.GeoJson(data=geo_j,
#                            style_function=lambda x: {'fillColor': 'orange'})
    if -0.01 < value < 0.01:
        geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 0.5,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0,
                                                                    })
    else:
        geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 0.5,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.8,
                                                                    })


    folium.Popup(f'{value:.3f}').add_to(geo_j)
    geo_j.add_to(m6)

In [ ]:
colormap = branca.colormap.linear.Reds_05.scale(0, max_value)
colormap = colormap.to_step(index=np.linspace(0, max_value, 1000))
colormap.caption = 'OHCA × Area Ratio not covered by AEDs and 119 stations'
colormap.add_to(m6)
m6